In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
import numpy as np
max_long = 25

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [7]:
!pip install unidecode

     |████████████████████████████████| 235 kB 13.4 MB/s 


In [8]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [9]:
import token_glove
token_glove.create_list, token_glove.token_ize, token_glove.voc_token, token_glove.dict_token, token_glove.token_tweet, token_glove.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [10]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [12]:
df = data.get_data(addon=True)
cleaned_df = data.clean_data(df)

In [13]:
text = token_glove.create_list(cleaned_df['text'])
token = token_glove.token_ize(text)
X = token_glove.token_tweet(text, token)

In [14]:
word_vector_matrix = token_glove.glove_vector(token)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [17]:
y = cleaned_df['target']

In [19]:
model = Sequential()

model.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model.add(LSTM(128,return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(32, return_sequences = False))
model.add(Dropout(0.4))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=[metrics.f1_m])
history = model.fit(X, y, batch_size=16, epochs = 7)

Epoch 1/7
1187/1187 [==============================] - 64s 51ms/step - loss: 0.4465 - f1_m: 0.5781
Epoch 2/7
1187/1187 [==============================] - 55s 46ms/step - loss: 0.3868 - f1_m: 0.6608
Epoch 3/7
1187/1187 [==============================] - 55s 46ms/step - loss: 0.3544 - f1_m: 0.6872
Epoch 4/7
1187/1187 [==============================] - 55s 47ms/step - loss: 0.3215 - f1_m: 0.7223
Epoch 5/7
1187/1187 [==============================] - 56s 47ms/step - loss: 0.2971 - f1_m: 0.7469
Epoch 6/7
1187/1187 [==============================] - 56s 47ms/step - loss: 0.2709 - f1_m: 0.7704
Epoch 7/7
1187/1187 [==============================] - 56s 47ms/step - loss: 0.2471 - f1_m: 0.7973


#Phase de Test

In [20]:
df = data.get_data(test=True)
test_df = data.clean_data(df)

In [21]:
text_test = token_glove.create_list(test_df['text'])
token_test = token_glove.token_ize(text_test)
X_test = token_glove.token_tweet(text_test, token_test)

In [23]:
results = model.predict(X_test)

In [33]:
results

array([[0.5386539 ],
       [0.17089102],
       [0.04549998],
       ...,
       [0.0556531 ],
       [0.41681018],
       [0.06549841]], dtype=float32)

In [24]:
def arrondi(n):
  if n >= 0.55:
    return 1
  else :
    return 0

In [25]:
to_submit = [arrondi(x[0]) for x in results]

In [26]:
to_submit_df = pd.DataFrame(to_submit)

In [27]:
to_submit_df.rename(columns={0: 'target'}, inplace=True)

In [28]:
to_submit_df['id'] = test_df.id

In [29]:
to_submit_df = to_submit_df.set_index('id')

In [31]:
to_submit_df.to_csv('/content/gdrive/My Drive/submission2.csv')

In [32]:
model.save('/content/gdrive/My Drive/RNN_camille_2CSV_v2')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/RNN_camille_2CSV_v2/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/RNN_camille_2CSV_v2/assets
